In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib.pyplot as plt

from imblearn.over_sampling import SMOTE
import gensim

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve, precision_score, recall_score
from sklearn.metrics import classification_report

from utilities import remove_empty_tweets
from sklearn.metrics import classification_report, f1_score


#text is already cleaned.
#assign cleaned data to these variables.
train_data_path = 'cleaned_data/cleaned_train_data_for_subtask1.csv'
test_data_path = 'cleaned_data/cleaned_test_data_for_subtask1.csv'
#read files.
train_data = pd.read_csv(train_data_path)
test_data = pd.read_csv(test_data_path)

print("Train set:"% train_data.columns, train_data.shape, len(train_data)) 
print("Test set:"% test_data.columns, test_data.shape, len(test_data)) 



C:\Users\Mena\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Train set: (20974, 7) 20974
Test set: (4997, 7) 4997


In [2]:
#remove empty tweets.
remove_empty_tweets(train_data, "#2_tweet_clean_V1")
remove_empty_tweets(test_data, "#2_tweet_clean_V1")

,#1_tweetid,#2_tweet,#3_country_label,#2_tweet_clean_V1,#2_tweet_clean_V2,#2_tweet_clean_V3,#classes_id
0,DEV_0,قولنا اون لاين لا يا علي اون لاين لا,Egypt,قولنا اون لاين لا يا علي اون لاين لا,قولنا اون لاين يا اون لاين,قولنا اون لاين يا,0
1,DEV_1,ههههه بايخه ههههه URL …,Oman,بايخه,بايخه,بايخه,6
2,DEV_2,ربنا يخليك يا دوك ولك المثل :D,Lebanon,ربنا يخليك يا دوك ولك المثل,ربنا يخليك يا دوك ولك المثل,ربنا يخليك يا دوك ولك المثل,8
3,DEV_3,#اوامر_ملكيه ياشباب اي واحد فيكم عنده شي يذكره...,Syria,اوامر ملكيه ياشباب اي واحد فيكم عنده شي يذكره ...,اوامر ملكيه ياشباب فيكم عنده شي يذكره بالعساف ...,اوامر ملكيه ياشباب فيكم عنده شي يذكره بالعساف ...,5
4,DEV_4,شد عالخط حتى هيا اكويسه,Libya,شد عالخط حتى هيا اكويسه,شد عالخط اكويسه,شد عالخط اكويسه,13
...,...,...,...,...,...,...,...
4992,DEV_4995,و في الشتاء مستحيل يقب,Libya,في الشتاء مستحيل يقب,الشتاء مستحيل يقب,الشتاء مستحيل يقب,13
4993,DEV_4996,اكيد حبيبتي وانتِ في كيفك وكيف طيبة قلبك,Libya,اكيد حبيبتي وانت في كيفك وكيف طيبه قلبك,اكيد حبيبتي وانت كيفك وكيف طيبه قلبك,اكيد حبيبتي وانت كيفك وكيف طيبه قلبك,13
4994,DEV_4997,يسعد صباح الناس رايقه,Jordan,يسعد صباح الناس رايقه,يسعد الناس رايقه,يسعد الناس رايقه,18
4995,DEV_4998,فرحتهم فيهاا ياتقبرنييي USER .twitter.com/hUS...,Morocco,فرحتهم فيها ياتقبرني,فرحتهم ياتقبرني,فرحتهم ياتقبرني,9


In [5]:
#prepare train and test data.
X_train = train_data['#2_tweet_clean_V1']
y_train = train_data['#classes_id']
X_test = test_data['#2_tweet_clean_V1']
y_test = test_data['#classes_id']


In [6]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_no = 50000
sql_len = 250
dim = 100

tokenizer = Tokenizer(num_words=max_no, lower=False)
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_tr = tokenizer.texts_to_sequences(X_train.values)
X_tr = pad_sequences(X_tr, maxlen=sql_len)
print('Shape of data tensor:', X_tr.shape)

tokenizer = Tokenizer(num_words=max_no, lower=False)
tokenizer.fit_on_texts(X_test.values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X_te = tokenizer.texts_to_sequences(X_test.values)
X_te = pad_sequences(X_te, maxlen=sql_len)
print('Shape of data tensor:', X_te.shape)

Found 49126 unique tokens.
Shape of data tensor: (20974, 250)
Found 17491 unique tokens.
Shape of data tensor: (4997, 250)


In [7]:
Y_tr = pd.get_dummies(y_train).values
print('Shape of label tensor:', Y_tr.shape)

Y_te = pd.get_dummies(y_test).values
print('Shape of label tensor:', Y_te.shape)

Shape of label tensor: (20974, 21)
Shape of label tensor: (4997, 21)


In [18]:
import keras
import tensorflow as tf
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa
import keras
import tensorflow
from keras.layers import Embedding, SpatialDropout1D, LSTM, Dense
#import tensorsflow_addons as tfa
vocab_size=50000
embedding_dim=64

model = tf.keras.Sequential([
    # Add an Embedding layer expecting input vocab of size 5000, and output embedding dimension of size 64 we set at the top
    tf.keras.layers.Embedding(vocab_size, embedding_dim),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(embedding_dim)),
#    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    # use ReLU in place of tanh function since they are very good alternatives of each other.
    tf.keras.layers.Dense(embedding_dim, activation='relu'),
    # Add a Dense layer with 6 units and softmax activation.
    # When we have multiple outputs, softmax convert outputs layers into a probability distribution.
    tf.keras.layers.Dense(6, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()
epochs = 10
batch_size = 64
history = model.fit(X_tr, Y_tr, epochs=epochs, batch_size=batch_size,validation_split=0.1)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 64)          3200000   
_________________________________________________________________
bidirectional_3 (Bidirection (None, 128)               66048     
_________________________________________________________________
dense_6 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_7 (Dense)              (None, 6)                 390       
Total params: 3,274,694
Trainable params: 3,274,694
Non-trainable params: 0
_________________________________________________________________


ValueError: A target array with shape (20974, 21) was passed for an output of shape (None, 6) while using as loss `categorical_crossentropy`. This loss expects targets to have the same shape as the output.

In [11]:
y_pred= model.predict(X_te)
y_pred = np.argmax(y_pred, axis=1)
y_test = test_data['#classes_id']
Y_te = pd.get_dummies(y_test).values

Y_te = np.argmax(Y_te, axis=1)
print(f1_score(Y_te, y_pred, average='macro'))

0.04607329650624178
